# Use Foundry REST API to chat with agents

```curl
# Optional Step: Create a conversation to use with the agent
curl -X POST https://YOUR-FOUNDRY-RESOURCE-NAME.services.ai.azure.com/api/projects/YOUR-PROJECT-NAME/openai/conversations?api-version=2025-11-15-preview \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $AZURE_AI_AUTH_TOKEN" \
  -d '{}'
# Lets say Conversation ID created is conv_123456789. Use this in the next step

#Chat with the agent to answer questions
curl -X POST https://YOUR-FOUNDRY-RESOURCE-NAME.services.ai.azure.com/api/projects/YOUR-PROJECT-NAME/openai/responses?api-version=2025-11-15-preview \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $AZURE_AI_AUTH_TOKEN" \
  -d '{
        "agent": {"type": "agent_reference", "name": "MyAgent"},
        "conversation" : "<YOUR_CONVERSATION_ID>",
        "input" : "What is the size of France in square miles?"
    }'

#Optional Step: Ask a follow-up question in the same conversation
curl -X POST https://YOUR-FOUNDRY-RESOURCE-NAME.services.ai.azure.com/api/projects/YOUR-PROJECT-NAME/openai/responses?api-version=2025-11-15-preview \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $AZURE_AI_AUTH_TOKEN" \
  -d '{
        "agent": {"type": "agent_reference", "name": "MyAgent"},
        "conversation" : "<YOUR_CONVERSATION_ID>",
        "input" : "And what is the capital city?"
    }'
```

In [1]:
resource_name = "engo-foundry"
project_name = "proj-default-2"
agent_name = "BasicAssistantAgent"
workflow_agent = "BasicWorkflow"

In [ ]:
# from azure.identity import InteractiveBrowserCredential

# credential = InteractiveBrowserCredential()
# scope = "https://ai.azure.com/.default"
# token = credential.get_token(scope)
# api_token = token.token

In [2]:
ai_azure_api_token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlBjWDk4R1g0MjBUMVg2c0JEa3poUW1xZ3dNVSIsImtpZCI6IlBjWDk4R1g0MjBUMVg2c0JEa3poUW1xZ3dNVSJ9.eyJhdWQiOiJodHRwczovL2FpLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2Q5NDFhODZkLTZjNjktNGZhMy04MTgxLTNjMjNlMGNiYTBmYy8iLCJpYXQiOjE3Njc2NDMxOTgsIm5iZiI6MTc2NzY0MzE5OCwiZXhwIjoxNzY3NjQ4MTEwLCJhY3IiOiIxIiwiYWlvIjoiQVVRQXUvOGFBQUFBWHVhamVrM3lsWThINEZiZm1VbXBTSDRzNlZabmNpOG1OY1V6UWhzMmtVbmZNNEhEY3ZITXF4Mk84Wlk5NTZ4UjJ0bzd3YjdxdlB4eXE1QlNCeU1FbVE9PSIsImFtciI6WyJwd2QiXSwiYXBwaWQiOiIyMGIzOGRlZi0xZjdlLTRmMGYtOWUzMC0xYjA5ZGQxYzgxMDgiLCJhcHBpZGFjciI6IjAiLCJmYW1pbHlfbmFtZSI6IkfDtnJsaXR6IiwiZ2l2ZW5fbmFtZSI6IkVucmljbyIsImdyb3VwcyI6WyI5Y2UzYzg3OC0wMmY1LTQyODMtOTZlOS02NDRiNTFkMzQ2ODUiLCI0ZTI2NmMwMi1kNzJkLTRmNjUtYmMyZS0xMDY0MzJkM2UwYjgiLCIyMjAzYjI1Yi0wY2M4LTQ0NmItYjAyOC1iZmIyMjJlNjQ3MjEiLCI0NjYyNGQ3Mi0wNzliLTQyMDMtYjU2MS1iMGQwYTJhZmI0ZGMiLCJjNWQwMmY3OC1jMzc5LTQ5ODktODYzNS1jNTg4MzljZjBjMWUiLCJiZjQwZWI4Zi0wODYxLTQ3OWQtOThmNS0wNWE2YTdjZjAxNzgiLCI4ZWEwZjVhMS0yMGE2LTRkMGUtODBhMi1kYzRiNWE2M2RlYTMiLCJlNDYzMTBlOC01M2RhLTRjNzctOWFjZC0yYzU0NDlkZTQ4ZmYiXSwiaWR0eXAiOiJ1c2VyIiwiaXBhZGRyIjoiMmEwMjo4MTA5OjlmMDg6YWQwMDo4ZGY4OmZlMDQ6ZTFkMDo3MjRkIiwibmFtZSI6IkVucmljbyBHw7ZybGl0eiIsIm9pZCI6IjRlNzZiMWJiLTM5YTEtNGU2Ny1iM2ZjLWM4NWUzY2IwYjliMCIsInB1aWQiOiIxMDAzMjAwNTJDRTJDRDM1IiwicmgiOiIxLkFUd0FiYWhCMldsc28wLUJnVHdqNE11Z19GOXZwaGpmMnhkTW5kY1dOSEVxbkw2N0FmTThBQS4iLCJzY3AiOiJ1c2VyX2ltcGVyc29uYXRpb24iLCJzaWQiOiIwMDEwMGQ3YS1iMGU0LTdiZWEtYjc4OC00ZmY2ODBjN2JmOWEiLCJzdWIiOiJhcURVUUVIVmF0RE5VNnAyVVZSSGdFZENsOWw5elNfMEVXYlZCd3kySjhrIiwidGlkIjoiZDk0MWE4NmQtNmM2OS00ZmEzLTgxODEtM2MyM2UwY2JhMGZjIiwidW5pcXVlX25hbWUiOiJlbnJpY28uZ29lcmxpdHpAcGNnbGFicy5vbm1pY3Jvc29mdC5jb20iLCJ1cG4iOiJlbnJpY28uZ29lcmxpdHpAcGNnbGFicy5vbm1pY3Jvc29mdC5jb20iLCJ1dGkiOiJPZDA4cWpwVS1rS256M3czS2RpSkFRIiwidmVyIjoiMS4wIiwieG1zX2FjdF9mY3QiOiIzIDkiLCJ4bXNfZnRkIjoiS0hWdXZDazZEYnRMNWhPTjZxY1B3MWNPbmZQaGxuT0pibWI4djI5ZnpPb0JaWFZ5YjNCbGQyVnpkQzFrYzIxeiIsInhtc19pZHJlbCI6IjEgMTAiLCJ4bXNfc3ViX2ZjdCI6IjMgMTQifQ.FIMd1j-3BeEG6-aYVLWBwg8Axi6OgtDCxZyNIgx6d59XpB04Rxu0Q_KI5QEiDbuXOB24wmA3EPXG33VnxUd4G50745U7oXrUxpANiMrqSowB2gkl4jVQjOV8BEMxb6w7nBClRWY66PrLqsmZ_oLvdJjI30Dje1lShnqoFcsR9jkCTvDZ8BDIPbfXZXRz3X6LBJ_-7xGZSUnVECit2aU41oxPExsn5Ivj_QS8DTD55S7nyftNLI0IB3aqFM7nVfrVLdLUhj5uoKzW36O-FuEb1GdcfitXYKraNgOQ_VurRptxIhCU2jF_3j-AXazcVY0h5RIdpVQYi-jAqHnAnM51zA"

In [3]:
# FoundryAPIClient for Azure Foundry REST API
import requests

class FoundryAPIClient:
    def __init__(self, resource_name, project_name, api_token):
        self.base_url = f"https://{resource_name}.services.ai.azure.com/api/projects/{project_name}/openai"
        # self.base_url = "https://engo-foundry.services.ai.azure.com/api/projects/proj-default-2"
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_token}"
        }

    def create_conversation(self):
        url = f"{self.base_url}/conversations?api-version=2025-11-15-preview"
        response = requests.post(url, headers=self.headers, json={})
        response.raise_for_status()
        return response.json()

    def send_message(self, agent_name, conversation_id, input_text):
        url = f"{self.base_url}/responses?api-version=2025-11-15-preview"
        payload = {
            "agent": {"type": "agent_reference", "name": agent_name},
            "conversation": conversation_id,
            "input": input_text
        }
        response = requests.post(url, headers=self.headers, json=payload)
        response.raise_for_status()
        return response.json()

    def send_message_streaming(self, agent_name, conversation_id, input_text):
        url = f"{self.base_url}/responses?api-version=2025-11-15-preview"
        payload = {
            "agent": {"type": "agent_reference", "name": agent_name},
            "conversation": conversation_id,
            "input": input_text
        }
        response = requests.post(url, headers=self.headers, json=payload, stream=True)
        response.raise_for_status()
        return response.iter_lines()

# Variables to fill in:

# Example usage:

In [4]:
client = FoundryAPIClient(resource_name, project_name, ai_azure_api_token)
conv = client.create_conversation()
conversation_id = conv.get('id')

In [5]:
conversation_id

'conv_83d5b1530568e56500DBvaVO1HYUc64c5gQD2l4dznkmc6uuMc'

In [7]:
response = client.send_message_streaming(agent_name, conversation_id, "Gib mir ein traditionelles meat ball rezept")
for line in response:
    print(line)

b'{"metadata":{},"temperature":1,"top_p":1,"user":null,"service_tier":"default","top_logprobs":0,"model":"gpt-4.1","reasoning":{},"background":false,"text":{"format":{"type":"text"},"verbosity":"medium"},"tools":[],"tool_choice":"auto","truncation":"disabled","id":"resp_83d5b1530568e56500695c1fddfd888190892a2b7f4067eaeb","object":"response","status":"completed","created_at":1767645150,"error":null,"incomplete_details":null,"output":[{"type":"message","id":"msg_83d5b1530568e56500695c1fde6d508190a410de7b6aa4c08a","status":"completed","role":"assistant","content":[{"type":"output_text","text":"Hier ist ein klassisches Rezept f\xc3\xbcr traditionelle Fleischb\xc3\xa4llchen (Meatballs):\\n\\n**Zutaten:**\\n- 500 g gemischtes Hackfleisch (Rind und Schwein)\\n- 1 Zwiebel, fein gehackt\\n- 2 Knoblauchzehen, gepresst oder fein gehackt\\n- 1 altbackenes Br\xc3\xb6tchen (oder 2 Scheiben Toastbrot)\\n- 100 ml Milch\\n- 1 Ei\\n- 2 EL gehackte Petersilie\\n- 50 g geriebener Parmesan (optional, typis

In [13]:
workflow_client = FoundryAPIClient(resource_name, project_name, ai_azure_api_token)
wf_conv = client.create_conversation()
wf_conversation_id = wf_conv.get('id')
wf_conversation_id

'conv_fb1b90f115285b9400f9bPPetfDLRt8tqbYiNQGwE3bIjMps65'

In [31]:
conversaion_msgs = [
    "hi",
    "My name is Teddy"
]

workflow_client = FoundryAPIClient(resource_name, project_name, ai_azure_api_token)
wf_conv = client.create_conversation()
wf_conversation_id = wf_conv.get('id')
wf_conversation_id

print("User:\t\t", conversaion_msgs[0])
response_1 = workflow_client.send_message(workflow_agent, wf_conversation_id, conversaion_msgs[0])
print("Ask Question:\t", response_1["output"][1]["content"][0]["text"])
print("User:\t\t", conversaion_msgs[1])
response_2 = workflow_client.send_message(workflow_agent, wf_conversation_id, conversaion_msgs[1])
print("Agent:\t\t", response_2["output"][1]["content"][0]["text"])
print("Send message:\t", response_2["output"][3]["content"][0]["text"])

User:		 hi
Ask Question:	 What is your name?
User:		 My name is Teddy
Agent:		 Hello, Teddy! How can I assist you today?
Send message:	 Conversation is ending here.


In [34]:
import json

# save responses to files
with open("resp_conversation.json", "w") as f:
    json.dump(wf_conv, f, indent=4)

with open("resp_response_1.json", "w") as f:
    json.dump(response_1, f, indent=4)

with open("resp_response_2.json", "w") as f:
    json.dump(response_2, f, indent=4)

In [26]:
response_2["output"]

[{'type': 'workflow_action',
  'id': 'wfa_5a488c84a019d28f004hnKRWJTk37e2q9xgcjOWz5yUAzNFpaF',
  'created_by': {'agent': {'type': 'agent_id',
    'name': 'BasicWorkflow',
    'version': '1'},
   'response_id': 'wfresp_5a488c84a019d28f00nrGWnTEnfEdYNjxHmF7e0quy6YcyGsEr'},
  'kind': 'InvokeAzureAgent',
  'action_id': 'action-1767634350800',
  'parent_action_id': 'trigger_wf',
  'previous_action_id': 'action-1767634216505_Post',
  'status': 'completed'},
 {'type': 'message',
  'id': 'msg_0b91fa69e464ca7800695c09e9402c81948c93036e892018e0',
  'created_by': {'agent': {'type': 'agent_id',
    'name': 'BasicAssistantAgent',
    'version': '2'}},
  'status': 'completed',
  'role': 'assistant',
  'content': [{'type': 'output_text',
    'text': "Hi Teddy! It's nice to meet you. How can I assist you today?",
    'annotations': [],
    'logprobs': []}]},
 {'type': 'workflow_action',
  'id': 'wfa_5a488c84a019d28f00zE4nKSZYxDNxLVT77xWxCr6vY8u9YyEW',
  'created_by': {'agent': {'type': 'agent_id',
   

In [22]:
response_1["output"][1]["content"][0]["text"]

'What is your name?'

In [11]:
response_2 = workflow_client.send_message(workflow_agent, wf_conversation_id, "My name is Teddy")
response_2

{'metadata': {},
 'temperature': None,
 'top_p': None,
 'user': None,
 'service_tier': 'auto',
 'model': '',
 'background': False,
 'tools': [],
 'tool_choice': 'auto',
 'truncation': 'auto',
 'id': 'wfresp_a3d7606552dd319600ujcGUFRPbcRrAe1Huwntfec3a8SjkYoX',
 'object': 'response',
 'status': 'completed',
 'created_at': 1767639155,
 'error': None,
 'incomplete_details': None,
 'output': [{'type': 'workflow_action',
   'id': 'wfa_a3d7606552dd319600NqBzvxHA9KuGwXvQD7HMnMY1POsBkRal',
   'created_by': {'agent': {'type': 'agent_id',
     'name': 'BasicWorkflow',
     'version': '1'},
    'response_id': 'wfresp_a3d7606552dd319600ujcGUFRPbcRrAe1Huwntfec3a8SjkYoX'},
   'kind': 'InvokeAzureAgent',
   'action_id': 'action-1767634350800',
   'parent_action_id': 'trigger_wf',
   'previous_action_id': 'action-1767634216505_Post',
   'status': 'completed'},
  {'type': 'message',
   'id': 'msg_0c2ba75db7ad4dd100695c0875b3ec8194aae3a363cce20776',
   'created_by': {'agent': {'type': 'agent_id',
     'n

In [13]:
import requests

class FoundryAPIClient:
    def __init__(self, resource_name, project_name, api_token):
        self.base_url = f"https://{resource_name}.services.ai.azure.com/api/projects/{project_name}/openai"
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {ai_azure_api_token}"
        }

    def create_conversation(self):
        url = f"{self.base_url}/conversations?api-version=2025-11-15-preview"
        r = requests.post(url, headers=self.headers, json={})
        r.raise_for_status()
        return r.json()

    def send_message(self, agent_name, conversation_id, input_text):
        url = f"{self.base_url}/responses?api-version=2025-11-15-preview"
        payload = {
            "agent": {"type": "agent_reference", "name": agent_name},
            "conversation": conversation_id,
            "input": input_text
        }
        r = requests.post(url, headers=self.headers, json=payload)
        r.raise_for_status()
        return r.json()

    def send_message_streaming(self, agent_name, conversation_id, input_text):
        url = f"{self.base_url}/responses?api-version=2025-11-15-preview"

        payload = {
            "agent": {"type": "agent_reference", "name": agent_name},
            "conversation": conversation_id,
            "input": input_text,
            "stream": True,  # <- DAS ist der entscheidende Schalter
        }

        headers = dict(self.headers)
        headers["Accept"] = "text/event-stream"  # hilft, damit wirklich SSE geliefert wird

        r = requests.post(url, headers=headers, json=payload, stream=True)
        r.raise_for_status()

        # wichtig: requests kann sonst recht stark puffern
        def _iter_sse_lines():
            for line in r.iter_lines(decode_unicode=True, chunk_size=1):
                if not line:
                    continue
                # SSE: üblicherweise "data: {...}"
                if line.startswith("data:"):
                    line = line[len("data:"):].lstrip()
                # Manche Streams senden Terminator
                if line == "[DONE]":
                    break
                yield line

        return _iter_sse_lines()

In [ ]:
client = FoundryAPIClient(resource_name, project_name, ai_azure_api_token)
conv = client.create_conversation()
stream = client.send_message_streaming(agent_name, conv["id"], "Gib mir ein traditionelles meatball rezept")

lines = []
for line in stream:
    print(line)
    lines.append(line)

with open("resp_agent_foundry_stream_output.txt", "w") as f:
    for line in lines:
        f.write(line + "\n")

event: response.created
{"type":"response.created","sequence_number":0,"response":{"id":"resp_42349b27fb4cda4000695c22bce9b08190939d8197cb6415b2","object":"response","created_at":1767645885,"status":"in_progress","background":false,"content_filters":null,"conversation":{"id":"conv_42349b27fb4cda4000ieyskLT7S9GEYedu1egnu2oT8M4BTcGH"},"error":null,"incomplete_details":null,"instructions":"You are an helpful assistant.","max_output_tokens":null,"max_tool_calls":null,"model":"gpt-4.1","output":[],"parallel_tool_calls":true,"previous_response_id":null,"prompt_cache_key":null,"prompt_cache_retention":null,"reasoning":{"effort":null,"summary":null},"safety_identifier":null,"service_tier":"auto","store":true,"temperature":1.0,"text":{"format":{"type":"text"},"verbosity":"medium"},"tool_choice":null,"tools":[],"top_logprobs":0,"top_p":1.0,"truncation":"disabled","usage":null,"user":null,"metadata":{},"agent":{"type":"agent_id","name":"BasicAssistantAgent","version":"2"}}}
event: response.in_pro

In [17]:
wf_client = FoundryAPIClient(resource_name, project_name, ai_azure_api_token)
wf_conv = wf_client.create_conversation()
stream_1 = wf_client.send_message_streaming(workflow_agent, wf_conv["id"], "Hi")

lines_1 = []
for line in stream_1:
    print(line)
    lines_1.append(line)

lines_2 = []
stream_2 = wf_client.send_message_streaming(workflow_agent, wf_conv["id"], "Mein name ist Teddy")

print("\n\n\n---\n\n\n")
for line in stream_2:
    print(line)
    lines_2.append(line)

with open("resp_agent_workflow_foundry_stream_1_output.txt", "w") as f:
    for line in lines_1:
        f.write(line + "\n")
with open("resp_agent_workflow_foundry_stream_2_output.txt", "w") as f:
    for line in lines_2:
        f.write(line + "\n")

event: response.created
{"type":"response.created","sequence_number":1,"response":{"metadata":{},"temperature":null,"top_p":null,"user":null,"service_tier":"auto","model":"","background":false,"tools":[],"tool_choice":"auto","truncation":"auto","id":"wfresp_79c5d0f2bd5934fc009E1dJX02WpzdSfDUuZ2CbRVLFBUPGllA","object":"response","status":"in_progress","created_at":1767646335,"error":null,"incomplete_details":null,"output":[],"instructions":"","parallel_tool_calls":true,"conversation":{"id":"conv_79c5d0f2bd5934fc00EFoPWT02gYjtDpX9d7q2oxcJSWdnVwF0"},"agent":{"type":"agent_id","name":"BasicWorkflow","version":"1"},"structured_inputs":{}}}
event: response.in_progress
{"type":"response.in_progress","sequence_number":2,"response":{"metadata":{},"temperature":null,"top_p":null,"user":null,"service_tier":"auto","model":"","background":false,"tools":[],"tool_choice":"auto","truncation":"auto","id":"wfresp_79c5d0f2bd5934fc009E1dJX02WpzdSfDUuZ2CbRVLFBUPGllA","object":"response","status":"in_progre